**New Hacker Rank Queries - Intermediate Level**

Weather Observation Station 5

<span style="color: rgb(87, 104, 113); font-family: OpenSans, Arial, Helvetica, sans-serif; background-color: rgb(255, 255, 255);">Query the two cities in&nbsp;</span> <span style="border: 0px; font-variant-numeric: inherit; font-variant-east-asian: inherit; font-weight: bolder; font-stretch: inherit; line-height: inherit; font-family: OpenSans, Arial, Helvetica, sans-serif; margin: 0px; outline: 0px; padding: 0px; vertical-align: baseline; color: var(--color-text-dark,#0e141e); background-color: rgb(255, 255, 255);">STATION</span> <span style="color: rgb(87, 104, 113); font-family: OpenSans, Arial, Helvetica, sans-serif; background-color: rgb(255, 255, 255);">&nbsp;with the shortest and longest&nbsp;</span> <span style="border: 0px; font-style: inherit; font-variant-numeric: inherit; font-variant-east-asian: inherit; font-stretch: inherit; line-height: inherit; font-family: OpenSans, Arial, Helvetica, sans-serif; margin: 0px; outline: 0px; padding: 0px; vertical-align: baseline; color: rgb(87, 104, 113); background-color: rgb(255, 255, 255);">CITY</span> <span style="color: rgb(87, 104, 113); font-family: OpenSans, Arial, Helvetica, sans-serif; background-color: rgb(255, 255, 255);">&nbsp;names, as well as their respective lengths (i.e.: number of characters in the name). If there is more than one smallest or largest city, choose the one that comes first when ordered alphabetically.</span>  
<span style="color: rgb(87, 104, 113); font-family: OpenSans, Arial, Helvetica, sans-serif; background-color: rgb(255, 255, 255);">The&nbsp;</span> <span style="border: 0px; font-variant-numeric: inherit; font-variant-east-asian: inherit; font-weight: bolder; font-stretch: inherit; line-height: inherit; font-family: OpenSans, Arial, Helvetica, sans-serif; margin: 0px; outline: 0px; padding: 0px; vertical-align: baseline; color: var(--color-text-dark,#0e141e); background-color: rgb(255, 255, 255);">STATION</span> <span style="color: rgb(87, 104, 113); font-family: OpenSans, Arial, Helvetica, sans-serif; background-color: rgb(255, 255, 255);">&nbsp;table is described as follows:</span>

In [ ]:
-- Using a WITH clause:
WITH minimum AS 
(   SELECT TOP 1 city, len(city) as city_length
    FROM station 
    ORDER BY city_length, city
), 
maximum AS
(   SELECT TOP 1 city, len(city) AS city_length 
    FROM station
    ORDER BY city_length DESC, city
)
SELECT * 
FROM minimum
UNION ALL 
SELECT * 
FROM maximum

In [ ]:
-- Using Pagination
SELECT city, LEN(city)
FROM station
ORDER BY LEN(city), city
OFFSET 0 ROWS
FETCH NEXT 1 ROWS ONLY
SELECT city, LEN(city)
FROM station
ORDER BY LEN(city) DESC, city
OFFSET 0 ROWS
FETCH NEXT 1 ROWS ONLY


In [ ]:
-- Using Subqueries
SELECT  TOP 1 city, LEN(city)
FROM    station
WHERE   LEN(city) = (   SELECT  MIN(LEN(city))
                        FROM    station
                    )
ORDER BY city;
SELECT  TOP 1 city, LEN(city)
FROM    station
WHERE   LEN(city) = (   SELECT  MAX(LEN(city))
                        FROM    station
                    )
ORDER BY city;

**Weather Station 20**

Find the Median value of the northern latitudes and round to 4 decimal places

In [ ]:
-- Using WITH Clauses
WITH bottom_50 AS
(   SELECT  TOP 50 PERCENT lat_n
    FROM    station
    ORDER BY lat_n
), 
top_50 AS
(   SELECT  TOP 50 PERCENT lat_n
    FROM    station
    ORDER BY lat_n DESC
)
SELECT CAST(
    ((   SELECT  MAX(lat_n)
        FROM    bottom_50
    ) 
    + 
     (  SELECT MIN(lat_n)
        FROM    top_50)
    )/2 AS DECIMAL(18, 4)) AS Median

In [ ]:
-- Using Subqueries in the FROM clauses
SELECT CAST(
    ((  SELECT  MAX(lat_n)
        FROM   (
                SELECT TOP 50 PERCENT lat_n
                FROM station
                ORDER BY lat_n
               ) AS bottom_50
    ) 
    + 
    (   SELECT MIN(lat_n)
        FROM  (
                SELECT TOP 50 PERCENT lat_n
                FROM station 
                ORDER BY lat_n DESC
              ) AS top_50   
     )
    )/2 AS DECIMAL(18, 4)) AS Median

In [ ]:
-- Using ROW_NUMBER() OVER() to find the rank and reverse ranks of the ordered set
WITH Ranks AS 
(   SELECT  LAT_N, 
            ROW_NUMBER() OVER(ORDER BY lat_n ASC) AS Rank, 
            ROW_NUMBER() OVER(ORDER BY lat_n DESC) AS Reverse_rank 
    FROM STATION 
) 
SELECT  CAST((AVG(LAT_N)) AS DECIMAL(10,4)) 
FROM    Ranks 
WHERE   ABS(Rank-Reverse_rank) <= 1;

In [ ]:
-- With iteration
DECLARE @counter INT;

SET @counter = (SELECT COUNT(*) 
                FROM station);

WITH cte AS 
( 
    SELECT  lat_n, 
            ROW_NUMBER() OVER (ORDER BY LAT_N) AS row_num 
    FROM station
)
SELECT FORMAT(AVG(lat_n),'.0000') 
FROM cte 
WHERE row_num BETWEEN (@counter+1)/2 AND @counter/2+1
        

# Ollivander's Wand Shop 

Harry Potter and his friends are at Ollivander's with Ron, finally replacing Charlie's old broken wand.

Hermione decides the best way to choose is by determining the minimum number of gold galleons needed to buy each non-evil wand of high power and age. Write a query to print the id, age, coins_needed, and power of the wands that Ron's interested in, sorted in order of descending power. If more than one wand has same power, sort the result in order of descending age.

In [ ]:
-- Using a subquery in the FROM clause
SELECT w.id,
       cheap.age,
       cheap.coins,
       cheap.power
FROM wands w,

    (SELECT w.code,
            p.age,
            w.power,
            MIN(coins_needed) AS coins
     FROM wands w
     INNER JOIN wands_property p ON w.code = p.code
     WHERE is_evil = 0
     GROUP BY w.code,
              p.age,
              w.power) AS cheap
WHERE w.code = cheap.code
    AND w.power = cheap.power
    AND w.coins_needed = cheap.coins
ORDER BY cheap.power DESC,
         cheap.age DESC

In [ ]:
-- Using a WITH clause to get the cheap and safe wands before joining the non-aggregate data

WITH cheap_and_safe AS
    (SELECT p.age,
            w.power,
            MIN(coins_needed) coins
     FROM wands w
     INNER JOIN wands_property p ON w.code = p.code
     WHERE p.is_evil = 0
     GROUP BY p.age,
              w.power)
SELECT w.id,
       p.age,
       w.coins_needed,
       w.power
FROM wands w
INNER JOIN wands_property p ON w.code = p.code
INNER JOIN cheap_and_safe ON w.power = cheap_and_safe.power
AND w.coins_needed = cheap_and_safe.coins
AND p.age = cheap_and_safe.age
ORDER BY cheap_and_safe.power DESC,
         cheap_and_safe.age DESC

In [ ]:
-- Using Row order and partitioning to only pick the top match for each power/cost
-- This removes the group by when partitioning by the power and age, then ranking them in order of high to low power and low to high cost
-- The top pick for the best wand would be the one with the highest power and lowest cost per power/age partition

WITH cheap_and_powerful AS
    ( SELECT w.id,
             p.age,
             w.coins_needed,
             w.power,
             ROW_NUMBER() OVER (PARTITION BY power, age
                                ORDER BY power DESC, coins_needed) AS best_wand
     FROM wands w
     JOIN wands_property p ON w.code = p.code
     WHERE p.is_evil = 0 )
SELECT id,
       age,
       coins_needed,
       power
FROM cheap_and_powerful
WHERE best_wand = 1
ORDER BY power DESC,
         age DESC;



In [ ]:
-- Using Rank() to get the cheapest wands 
WITH Cheap_Wands AS
    ( SELECT w.id,
             p.age,
             w.coins_needed,
             w.power,
             RANK() OVER (PARTITION BY power,
                                       age
                          ORDER BY coins_needed) AS cheapest
     FROM wands w
     JOIN wands_property p ON w.code = p.code
     WHERE p.is_evil = 0 )
SELECT id,
       age,
       coins_needed,
       power
FROM Cheap_Wands
WHERE cheapest = 1
ORDER BY power DESC,
         age DESC;

